# 01 — Download & Inspect Data (GSE77314)

Este notebook descarga los datos del estudio **GSE77314** desde GEO, organiza los archivos crudos y extrae la metadata necesaria para clasificar las muestras entre **Normal** y **Tumor**.

Los archivos procesados se almacenan en:
- `data/raw/gse77314/`
- `data/processed/gse77314/`

Este notebook forma parte del pipeline:
1. **01_download_data.ipynb** ← *ESTE*
2. 02_preprocessing.ipynb
3. 03_EDA.ipynb
4. 04_dimensionallity_reduction.ipynb
5. 05_ml_models.ipynb
6. 06_interpretation.ipynb

In [ ]:
# Celda 1 — Imports y paths
import os
import GEOparse
import pandas as pd

RAW_DIR = "data/raw/gse77314"
PROC_DIR = "data/processed/gse77314"

os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)

print("Directorios listos:")
print(RAW_DIR)
print(PROC_DIR)

## Descargar GSE77314 desde GEO (SOFT file)
Si el archivo ya existe en el repositorio, esta celda no lo volverá a descargar.


In [ ]:
# Celda 2 — Descargar SOFT si no existe
soft_path = os.path.join(RAW_DIR, "GSE77314_family.soft.gz")

if not os.path.exists(soft_path):
    print("Descargando GSE77314...")
    gse = GEOparse.get_GEO(geo="GSE77314", destdir=RAW_DIR)
else:
    print("SOFT ya descargado.")

## Cargar el SOFT y explorar metadata de las muestras
Aquí extraemos el campo **source_name_ch1** para identificar si una muestra es *Normal* o *Tumor*.

In [ ]:
# Celda 3 — Leer SOFT
gse = GEOparse.get_GEO(filepath=soft_path)

print(f"Total de muestras: {len(gse.gsms)}")

for gsm_name, gsm in list(gse.gsms.items())[:5]:
    print(gsm_name, gsm.metadata.get('source_name_ch1', []))

## Extraer etiquetas (Normal vs Tumor)
Las etiquetas se encuentran dentro de `source_name_ch1` con textos como:
- `diagnosis: Normal`
- `diagnosis: Tumor`

In [ ]:
# Celda 4 — Construir diccionario sample -> label
labels = {}

for gsm_name, gsm in gse.gsms.items():
    src = gsm.metadata.get("source_name_ch1", [])
    src = " ".join(src).lower() if isinstance(src, list) else str(src).lower()

    if "normal" in src:
        labels[gsm_name] = 0
    elif "tumor" in src:
        labels[gsm_name] = 1
    else:
        labels[gsm_name] = None


labels_df = pd.DataFrame.from_dict(labels, orient="index", columns=["label"])
labels_df.to_csv(os.path.join(PROC_DIR, "gse77314_labels.csv"))

labels_df.value_counts(dropna=False)

## Procesar el archivo `GSE77314_differential_genes.xlsx`
Este archivo contiene los genes diferenciales entre Normal/Tumor.


In [ ]:
# Celda 5 — Procesar differential_genes.xlsx
xlsx_path = os.path.join(RAW_DIR, "GSE77314_differential_genes.xlsx")

df = pd.read_excel(xlsx_path)
df.to_csv(os.path.join(PROC_DIR, "gse77314_differential_genes.csv"), index=False)

df.head(), df.shape

# Fin del notebook
Los datos están listos para continuar con:

➡️ **02_preprocessing.ipynb**
